In [1]:
import time, random, numpy as np, pandas as pd
from flask import Flask, request, jsonify, render_template_string
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from textblob import TextBlob
import threading

app = Flask(__name__)

def generate_customer_data(n=100):
    X = np.random.randint(18, 65, size=(n, 3))
    y = (X[:, 2] > 50).astype(int)
    return X, y

def simulate_iot_data_stream():
    return {
        "temperature": round(random.uniform(20.0, 25.0), 2),
        "foot_traffic": random.randint(50, 200),
        "humidity": round(random.uniform(30.0, 50.0), 2),
    }

def train_model():
    X, y = generate_customer_data(500)
    model = RandomForestClassifier()
    model.fit(X, y)
    return model

def chatbot_response(product):
    return f"Hey! Because you're interested in {product}, here's a custom deal just for you!"

customer_data = {
    'user_id': [1, 2, 3, 4, 5],
    'feature_1': [0.1, 0.2, 0.2, 0.4, 0.9],
    'feature_2': [0.3, 0.6, 0.7, 0.1, 0.8],
    'feature_3': [0.4, 0.8, 0.5, 0.2, 0.3]
}
df = pd.DataFrame(customer_data)
recommender = NearestNeighbors(metric='cosine', algorithm='brute')
recommender.fit(df.drop(['user_id'], axis=1))

feedback_log = []

@app.route("/")
def dashboard():
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head><title>Marketing System</title></head>
    <body>
        <h1>Personalized Marketing Dashboard</h1>
        <h3>Try Product Chatbot</h3>
        <input id="prod" placeholder="Product name" />
        <button onclick="askBot()">Ask</button>
        <p id="botRes"></p>

        <h3>Analyze Feedback</h3>
        <input id="textInput" placeholder="Your feedback" />
        <button onclick="analyze()">Check</button>
        <p id="sentimentRes"></p>

        <h3>Get Similar Users</h3>
        <button onclick="recommend()">Recommend</button>
        <p id="recs"></p>

        <script>
            function askBot() {
                let val = document.getElementById("prod").value;
                fetch("/chatbot", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ product: val })
                }).then(r => r.json()).then(d => {
                    document.getElementById("botRes").innerText = d.response;
                });
            }

            function analyze() {
                let text = document.getElementById("textInput").value;
                fetch("/sentiment", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ text: text })
                }).then(r => r.json()).then(d => {
                    document.getElementById("sentimentRes").innerText = "Sentiment: " + d.sentiment;
                });
            }

            function recommend() {
                fetch("/recommend", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ vector: [0.2, 0.5, 0.8] })
                }).then(r => r.json()).then(d => {
                    document.getElementById("recs").innerText = "Similar users: " + d.recommendations.join(", ");
                });
            }
        </script>
    </body>
    </html>
    """)

@app.route("/chatbot", methods=["POST"])
def chatbot():
    data = request.json
    reply = chatbot_response(data["product"])
    return jsonify({"response": reply})

@app.route("/sentiment", methods=["POST"])
def sentiment():
    text = request.json["text"]
    polarity = TextBlob(text).sentiment.polarity
    sentiment = "positive" if polarity > 0.1 else "negative" if polarity < -0.1 else "neutral"
    feedback_log.append({"text": text, "sentiment": sentiment})
    return jsonify({"sentiment": sentiment})

@app.route("/recommend", methods=["POST"])
def recommend():
    vector = request.json["vector"]
    d, i = recommender.kneighbors([vector], n_neighbors=3)
    recs = df.iloc[i[0]]['user_id'].tolist()
    return jsonify({"recommendations": recs})

@app.route("/environment")
def environment():
    return jsonify(simulate_iot_data_stream())

def run_app():
    app.run(port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/May/2025 19:41:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:07] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:08] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:14] "POST /sentiment HTTP/1.1" 200 -
/home/mak/Documents/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
127.0.0.1 - - [12/May/2025 19:41:16] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:24] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:30] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:35] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:36] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:36] "POST /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 19:41:38] "POST /sent